# 根据 输入长度 选择示例

In [7]:
# 根据输入的提示词长度综合计算最终长度，智能截取或者添加提示词
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 假设已经有这么多的提示词示例组：
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"angry","output":"calm"},
  {"input":"calm","output":"angry"},
  {"input":"excited","output":"bored"},
  {"input":"bored","output":"excited"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
  input_variables=["input", "output"],
  template="原词: {input}\n反义词: {output}"
)

# 调用示例选择器
example_selector = LengthBasedExampleSelector(
  examples=examples,                 # 传入提示词示例组
  example_prompt=example_prompt,     # 设置提示词模板
  max_length=25,                     # 设置格式化后最大长度
  min_length=10                      # 设置格式化后最小长度
)

# 使用小样本提示词模板来实现动态示例的使用
dynamic_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="请给出一个单词的反义词",
  suffix="原词: {input}\n反义词: {output}",
  input_variables=["input", "output"]
)

In [8]:
# 小样本获得所有示例
print(dynamic_prompt.format(input="big", output=""))

请给出一个单词的反义词

原词: happy
反义词: sad

原词: sad
反义词: happy

原词: angry
反义词: calm

原词: calm
反义词: angry

原词: excited
反义词: bored

原词: big
反义词: 


In [9]:
# 如果输入长度很长，则最终输出会根据长度要求减少
long_string = "big and huge and mass and large and enormous and gigantic and colossal"
print(dynamic_prompt.format(input=long_string, output=""))

请给出一个单词的反义词

原词: happy
反义词: sad

原词: sad
反义词: happy

原词: big and huge and mass and large and enormous and gigantic and colossal
反义词: 


In [12]:
! pip install titoken
! pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement titoken (from versions: none)
ERROR: No matching distribution found for titoken


# 根据 输入相似度 选择示例（最大边际相关性）

· MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
· MMR会首先找出与输入最相似（即余弦相似度最大）的样本
· 然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本进行惩罚
· MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
· 关注如何在相关性和多样性之间找到一个平衡

In [34]:
# 使用MMR来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.prompts import FewShotPromptTemplate

import os

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

# 假设已经有这么多的提示词示例组：
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"angry","output":"calm"},
  {"input":"calm","output":"angry"},
  {"input":"excited","output":"bored"},
  {"input":"bored","output":"excited"},
  {"input":"高兴","output":"悲伤"},
  {"input":"开心","output":"难过"},
]

# 构造提示词模板
example_prompt = PromptTemplate(
  input_variables=["input", "output"],
  template="原词: {input}\n反义词: {output}"
)

In [35]:
# 引入自己的嵌入模型
dash_embedding = DashScopeEmbeddings(
    model="text-embedding-v3",dashscope_api_key=DASHSCOPE_API_KEY
)

# 调用MMR示例选择器
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
  examples,                 # 传入提示词示例组
  dash_embedding,           # 设置嵌入模型
  FAISS,                  # 设置向量数据库
  k=2,                              # 设置返回的示例数量
)

# 使用小样本提示词模板来实现动态示例的使用
mmr_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="请给出一个单词的反义词",
  suffix="原词: {adjective}\n反义词:",
  input_variables=["adjective"]
)

In [38]:
#  当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一堆示例组来填充提示词模板
print(mmr_prompt.format(adjective="伤心"))


请给出一个单词的反义词

原词: 开心
反义词: 难过

原词: calm
反义词: angry

原词: 伤心
反义词:


# 根据输入相似度选择示例（最大余弦相似度）
· 一种常见的相似度计算方法
· 它通过计算两个向量（在这里，向量可以表示文本、句子或词语）之间的余弦值来衡量他们的相似度
· 余弦值越接近1，表示两个向量越相似
· 主要关注的是如何准确衡量两个向量的相似度

In [40]:
# 使用最大余弦相似度来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

example_prompt = PromptTemplate(
  input_variables=["input", "output"],
  template="原词: {input}\n反义词: {output}"
)

# 假设已经有这么多的提示词示例组：
examples = [
  {"input":"happy","output":"sad"},
  {"input":"sad","output":"happy"},
  {"input":"angry","output":"calm"},
  {"input":"calm","output":"angry"},
  {"input":"excited","output":"bored"},
  {"input":"bored","output":"excited"},
  {"input":"高兴","output":"悲伤"},
  {"input":"开心","output":"难过"},
]

# 引入自己的嵌入模型
dash_embedding = DashScopeEmbeddings(
    model="text-embedding-v3",dashscope_api_key=DASHSCOPE_API_KEY
)

# 调用MMR示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
  examples,                 # 传入提示词示例组
  dash_embedding,           # 设置嵌入模型
  Chroma,                  # 设置向量数据库
  k=1,                              # 设置返回的示例数量
)

# 使用小样本提示词模板来实现动态示例的使用
similar_prompt = FewShotPromptTemplate(
  example_selector=example_selector,
  example_prompt=example_prompt,
  prefix="请给出一个单词的反义词",
  suffix="原词: {adjective}\n反义词:",
  input_variables=["adjective"]
)

In [41]:
print(similar_prompt.format(adjective="伤心"))

请给出一个单词的反义词

原词: 开心
反义词: 难过

原词: 伤心
反义词:
